In [7]:
import pandas as pd

import mysql.connector

# Database connection details
db_config = {
    'user': 'root',
    'password': '123456',
    'host': 'localhost',
    'database': 'tp_bd'
}

# Establish a database connection
conn = mysql.connector.connect(**db_config)
cursor_select = conn.cursor()
cursor_new_select = conn.cursor()
cursor2 = conn.cursor()
cursor = conn.cursor()


In [5]:
import json
from datetime import datetime

# Path to the JSON file
current_time = datetime.now().time()

json_file_path = 'lote_completo_ languages.json'
print("Opening JSON file ", current_time)
# Read the JSON file line by line
with open(json_file_path, 'r', encoding='utf-8') as file:
    current_time = datetime.now().time()
    print( "Reading JSON file ",current_time )
    # start from line 723879
     # Skip the first 723878 lines
    for _ in range(1331929):
        next(file)


    for line in file:
         
        record = json.loads(line.strip())
        repo_name = record['repo_name']
      # please look for the repo_name in the table repo. If it already exists,
      # get the id, otherwise insert it and get the id
        cursor_select.execute("SELECT id FROM repo WHERE name = %s", (repo_name,))
        result = cursor_select.fetchone()
        if result is None:
            insert_query = """
            INSERT INTO repo (name)
            VALUES (%s);
            """
            cursor_select.execute(insert_query, (repo_name,))
            conn.commit()
            cursor_select.execute("SELECT id FROM repo WHERE name = %s", (repo_name,))
            result = cursor_select.fetchone()
            repo_id = result[0]
        for language in record['language']:
            language_name = language['name']
            bytes_count = language['bytes']
            insert_query = """
            INSERT INTO languages (repo_id, language_name, language_bytes)
            VALUES (%s, %s, %s);
            """
            
            cursor.execute(insert_query, (repo_id, language_name, bytes_count))
            #print(f"Repository Name: {repo_name}, Language: {language_name}, Bytes: {bytes_count}")
         
            conn.commit() 
#conn.commit()
current_time = datetime.now().time()
print("Closing JSON file ", current_time)
cursor.close()
conn.close()





Opening JSON file  09:17:00.779576
Reading JSON file  09:17:00.781569


NameError: name 'repo_id' is not defined

In [3]:
import json
import mysql.connector
from datetime import datetime

# Database connection parameters
db_config = {
    'user': 'root',
    'password': '123456',
    'host': 'localhost',
    'database': 'tp_bd'
}

# Establish a connection to the database
conn = mysql.connector.connect(**db_config)
cursor_select = conn.cursor()
cursor_insert = conn.cursor()
cursor_update = conn.cursor()

# Path to the JSON file
json_file_path = 'commits-junio2022-en-adelante.json'
current_time = datetime.now().time()
print("Opening JSON file ", current_time)

# Read the JSON file line by line
with open(json_file_path, 'r', encoding='utf-8') as file:
    current_time = datetime.now().time()
    print("Reading JSON file ", current_time)

    for line in file:
        record = json.loads(line.strip())
        repo_name = record['repo_name'][0]
        name = record['name']
        email = record['email']
            
        # Look for the repo_name in the table repo
        cursor_select.execute("SELECT id FROM repo WHERE name = %s", (repo_name,))
        result = cursor_select.fetchone()
            
        # If there is a record
        if result is not None:
                
                try:
                    insert_query = """
                    INSERT INTO users (name, email) 
                    VALUES (%s, %s);
                    """
                    cursor_insert.execute(insert_query, (name, email))
                    conn.commit()
                    last_inserted_id = cursor_insert.lastrowid
                except mysql.connector.IntegrityError as err:
                    # Check if the error is due to a duplicate key
                    if err.errno == 1062:  # ER_DUP_ENTRY (duplicate entry)
                        # Get the user_id of the duplicate record
                        cursor_select.execute("SELECT id FROM users WHERE email = %s", (email,))
                        result = cursor_select.fetchone()
                        last_inserted_id = result[0]
                
                
                
                # Update the repo with the author_id
                update_query = """
                UPDATE repo
                SET author_id = %s
                WHERE name = %s;
                """
                cursor_update.execute(update_query, (last_inserted_id, repo_name))
                conn.commit()
                

# Commit the transaction and close the cursor and connection
conn.commit()
cursor_select.close()
cursor_insert.close()
cursor_update.close()
conn.close()


Opening JSON file  18:52:54.930676
Reading JSON file  18:52:54.930676


IntegrityError: 1062 (23000): Duplicate entry '6aeac71607ae24deafdc4e54fa256ea7529feec8@aosc.xyz' for key 'users.email_unique'

In [9]:
import json

# Path to the JSON file
json_file_path = 'commits-junio2022-en-adelante.json'

# Read the JSON file and print the first record
with open(json_file_path, 'r', encoding='utf-8') as file:
    first_line = file.readline().strip()
    first_record = json.loads(first_line)
    print("First Record:", first_record)

First Record: {'name': 'Victor Rudometov', 'email': '5063180d8c3ea1e4e8806e8a5622cc79772e2195@openjdk.org', 'subject': '8281296: Create a regression test for JDK-4515999', 'message': '8281296: Create a regression test for JDK-4515999\n\nBackport-of: f5d8cebbb6f1b38247c3b30ba8859874a0e98115\n', 'repo_name': ['openjdk/jdk11u'], 'seconds': '1665520820'}
